In [1]:
import pandas as pd
from datetime import date

In [2]:
N_CICLOS = 3
BIN_INI = 0b1
# Las definitivas deben de ir en el orden de los ciclos. Es decir, primero debe de ir el del ciclo 1, luego 2...
# El formato es el generado por el script de generación de notas definitivas del ciclo
REINGRESADOS = [
    2222985,
    2220048,
    2220040
]
DEFINITIVAS = [
    'DEFINITIVAS_CICLO_1_2022-09-26.xlsx',
    'DEFINITIVAS_CICLO_2_2022-09-26.xlsx',
    'DEFINITIVAS_CICLO_3_2022-09-29.xlsx'
]
CANCELADOS = pd.read_excel('./INPUT/DEFINITIVAS/ESTUDIANTES A MATRICULAR CICLO 4.xlsx', engine = 'openpyxl', sheet_name='DESERTORES REPORTADOS')['CODIGO_UIS'].unique()

In [3]:
def bin2strFix(n):
    bStr = bin(n)
    bStr = bStr[2:]
    lenStr = len(bStr)
    if lenStr < N_CICLOS:
        diff = N_CICLOS - lenStr
        for _ in range(diff):
            bStr = '0' + bStr
    return bStr
def strBin2Bool(strBin):
    rta = []
    for c in strBin:
        if c == '1':
            rta.append(True)
        else:
            rta.append(False)
    return rta
def strBin2Nombre(strBin):
    i = 1
    rta = ''
    for c in strBin:
        if c == '1':
            rta += 'Ciclo' + str(i) + '_'
        i += 1
    return rta

In [4]:
est = {
    'CÓDIGO': [],
    'NOMBRE': [],
    'APELLIDO': []
}
datos = []
for i in range (1, N_CICLOS + 1):
    est['NIV_CICLO_' + str(i)] = []
    datos.append(pd.read_excel('./INPUT/DEFINITIVAS/' + DEFINITIVAS[i - 1], engine='openpyxl'))

In [5]:
# Itera sobre el último dataframe
nAnteriores = len(DEFINITIVAS) - 2
for index, row in datos[-1].iterrows():
    if row['CÓDIGO'] in CANCELADOS and not (row['CÓDIGO'] in REINGRESADOS):
        continue
    est['CÓDIGO'].append(row['CÓDIGO'])
    est['NOMBRE'].append(row['NOMBRE'])
    est['APELLIDO'].append(row['APELLIDO'])
    est['NIV_CICLO_' + str(nAnteriores + 2)].append(row['REPROBÓ_SA'])
    for i in range(nAnteriores + 1):
        df = datos[i]
        df = df[df['CÓDIGO'] == row['CÓDIGO']]
        est['NIV_CICLO_' + str(i + 1)].append(df['REPROBÓ_SA'].iloc[0])

In [6]:
CONSOLIDADO = pd.DataFrame(est)
binI = BIN_INI
rtas = dict()
for _ in range(2**N_CICLOS - 1):
    strBin = bin2strFix(binI)
    combinacion = strBin2Bool(strBin)    
    dfRes = CONSOLIDADO
    for i in range(N_CICLOS):
        dfRes = dfRes[dfRes['NIV_CICLO_' + str(i + 1)] == combinacion[i]]
    if len(dfRes) > 0:
        rtas[strBin2Nombre(strBin)] = dfRes[['CÓDIGO', 'NOMBRE', 'APELLIDO']]
    binI += 1

In [7]:
writer = pd.ExcelWriter('./OUTPUT/MATRICULAS_NIV/MatriculasCursosNivelaciones_' + str(date.today()) + '.xlsx', engine='openpyxl')
temp = []
for item, value in rtas.items():
    temp.append(value)
    value.to_excel(writer, sheet_name = item, index = False) 
pd.concat(temp).to_excel(writer, sheet_name = 'CONSOLIDADO', index = False)
writer.close()
writer.handles = None
CONSOLIDADO.to_excel('./OUTPUT/MATRICULAS_NIV/NivelacionBinario_'+ str(date.today()) +'.xlsx', index = False)